# Packing CAT Cell demo notebook
The main functions of this demo are located in :
- [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)
- [nlc_optimisation.py](../irregular_object_packing/packing/nlc_optimisation.py)


### Contents
1. 
2. 

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```




In [1]:
# IMPORTS
import numpy as np
import pyvista as pv
import trimesh
from tqdm import tqdm

import sys
sys.path.append('../irregular_object_packing/')
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume, translation_matrix
from irregular_object_packing.mesh.utils import print_mesh_info
from irregular_object_packing.packing.chordal_axis_transform import (
    compute_cat_cells,
    face_coord_to_points_and_faces,
)
from irregular_object_packing.packing.nlc_optimisation import constraints_from_dict, objective, transform_v
from irregular_object_packing.packing.initialize import create_packed_scene, place_objects, save_image
from irregular_object_packing.packing.plots import create_plot

## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The yellow faces are the CAT faces.

In [2]:
from irregular_object_packing.packing.chordal_axis_transform import main
main()

KeyError: 1

## Demo computing CAT cells for Blood Cells in a cylindrical container
The cell below shows a demo of computing cat faces for a cylindrical container and multiple blood cell meshes. The yellow faces are the CAT faces and for each red blood cell there is a corresponding CAT cell. 



In [ ]:
mesh_volume = 0.1
container_volume = 10
coverage_rate = 0.3

DATA_FOLDER = "../data/mesh/"
loaded_mesh = trimesh.load_mesh(DATA_FOLDER + "RBC_normal.stl")
print_mesh_info(loaded_mesh, "loaded mesh")
# trimesh.Scene([loaded_mesh]).show()
# Scale the mesh to the desired volume
original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)
print_mesh_info(original_mesh, "scaled mesh")

container = trimesh.primitives.Sphere(radius=1, center=[0,0,0])
# container = trimesh.primitives.Cylinder(radius=1, height=1)
print_mesh_info(container, "original container")

container = scale_to_volume(container, container_volume)
print_mesh_info(container, "scaled container")

In [ ]:
# Initial placement of the objects
objects_coords = place_objects(container, original_mesh, coverage_rate=coverage_rate, c_scale=0.9)


In [ ]:
# get vertices of the object meshes and the container
# we resample to simplify and get a more uniform distribution of points
mesh = trimesh.sample.sample_surface_even(original_mesh, 1000)[0]
down_scale = 0.1 # initial downscale 
obj_points = []
tf_matrices = []
for i in range(len(objects_coords)):
    # get the object
    object = mesh.copy()
    # random rotation
    M_rot = trimesh.transformations.random_rotation_matrix()
    M_f = trimesh.transformations.scale_matrix(down_scale**(1/3))
    M_t = trimesh.transformations.translation_matrix(objects_coords[i] - np.array([0, 0, 0]))

    M = M_t @ M_f @ M_rot
    # first scale, then rotate, then translate
    points = trimesh.transform_points(object, M)

    tf_matrices.append(M)
    obj_points.append(points)

container_points = trimesh.sample.sample_surface_even(container, 10000)[0]

In [ ]:
# compute the cat cells for each object (takes approx. 1 min)
data = compute_cat_cells(obj_points, container_points)


## improvement
transformation optimisation will probably be local to the origin for each mesh. What we can do is use the object_coords to transform the cat cell to the origin, then optimise the transformation, then transform the cat cell back to the object_coords.  

Currently, first it needs to be confirmed that the transformation is local to the origin and that the translation from nlc_optimisation is correct. 

In [ ]:
from irregular_object_packing.packing.nlc_optimisation import constraints_from_dict, objective, transform_v
# for object 0:
    # Define the bounds for the variables
k = 2

# all_k_faces = cat_cells[k].pop("all")
r_bound = (-1 / 12 * np.pi, 1 / 12 * np.pi)
t_bound = (0, None)
bounds = [(0.1, None), r_bound, r_bound, r_bound, t_bound, t_bound, t_bound]
x0 = np.array([0.1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

# init_res = constraints_from_dict(x0, cat_cells[k])

constraint_dict = {"type": "ineq", "fun": constraints_from_dict, "args": (data.cat_cells[k],)}
res = minimize(objective, x0, method="SLSQP", bounds=bounds, constraints=constraint_dict)



In [ ]:
# This next part of code is for the visualization of the CAT cells.
# it processes the output of compute_cat_cells to create a PyVista mesh for each object
# and it applies the corresponding transformations on the object meshes.
from irregular_object_packing.packing.nlc_optimisation import construct_transform_matrix

object_meshes = []
cat_meshes = []
lim = len(data.cat_cells.keys()) 
# for k, v in tqdm(cat_cells.items()):
    # if k >= lim - 1:
    #     break
cat_points, poly_faces = face_coord_to_points_and_faces(data.cat_cells[k])
polydata = pv.PolyData(cat_points, poly_faces)

object_mesh = original_mesh.copy()
object_mesh = object_mesh.apply_transform(tf_matrices[k])
post_mesh = object_mesh.copy()

post_mesh.vertices = trimesh.transformations.transform_points(
    post_mesh.vertices, construct_transform_matrix(res.x)
)



In [ ]:
# create_plot(objects_coords, object_meshes, cat_meshes, container.to_mesh())
import pyvista as pv

# create the first plot
plotter = pv.Plotter(shape='1|1')  # replace with the filename/path of your first mesh
plotter.subplot(0)
plotter.add_title("Initial Placement")
plotter.add_mesh(object_mesh, color="red", opacity=0.8)
plotter.add_mesh(polydata, color="yellow", opacity=0.4)

# create the second plot
# plot2 = pv.Plotter()
plotter.subplot(1)
plotter.add_title("Optimized Placement")
plotter.add_mesh(post_mesh, color="red", opacity=0.8)
plotter.add_mesh(polydata, color="yellow", opacity=0.4)

# set up the rendering window with two horizontal subplots
# plotter.add_subplot(plot1, render=False)
# plotter.add_subplot(plot2, render=False)

# show the plot
plotter.show()


In [ ]:
res.x